In [1]:
import tensorflow as tf
import numpy as np

Step 1: Configure parameters here

In [2]:
num_clients = 48

Download shakespeare dataset

In [3]:
!rm -r leaf/shakespeare/data/*
!cd leaf/shakespeare && ./preprocess.sh -s niid --sf 0.2 -k 0 -t sample -tf 0.8

--2022-01-12 16:38:00--  http://www.gutenberg.org/files/100/old/1994-01-100.zip
Resolving www.gutenberg.org (www.gutenberg.org)... 2610:28:3090:3000:0:bad:cafe:47, 152.19.134.47
Connecting to www.gutenberg.org (www.gutenberg.org)|2610:28:3090:3000:0:bad:cafe:47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/files/100/old/1994-01-100.zip [following]
--2022-01-12 16:38:00--  https://www.gutenberg.org/files/100/old/1994-01-100.zip
Connecting to www.gutenberg.org (www.gutenberg.org)|2610:28:3090:3000:0:bad:cafe:47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2035857 (1,9M) [application/zip]
Saving to: ‘1994-01-100.zip’

1994-01-100.zip     100%[===================>]   1,94M  1,58MB/s    in 1,2s    

2022-01-12 16:38:03 (1,58 MB/s) - ‘1994-01-100.zip’ saved [2035857/2035857]

Archive:  1994-01-100.zip
  inflating: 100.txt                 
dividing txt data between users
Splitting .txt data between users
Di

In [4]:
!cd leaf/shakespeare && ./stats.sh

####################################
DATASET: shakespeare
1129 users
4226158 samples (total)
3743.28 samples per user (mean)
num_samples (std): 6212.26
num_samples (std/mean): 1.66
num_samples (skewness): 3.35
0 	 705
2000 	 126
4000 	 72
6000 	 56
8000 	 38
10000 	 33
12000 	 31
14000 	 16
16000 	 8
18000 	 11


Step 2:

In [5]:
from leaf_loader import load_leaf_dataset, process_text_input_indices, process_char_output_indices

users, train_data, test_data = load_leaf_dataset("shakespeare")

x_train = [process_text_input_indices(train_data[user]['x']) for user in users]
y_train = [process_char_output_indices(train_data[user]['y']) for user in users]

x_train = np.concatenate(x_train)
y_train = np.concatenate(y_train)

Step 3: Save the results.

In [6]:
# Sort IID
perms = np.random.permutation(x_train.shape[0])
x_train, y_train = x_train[perms, :], y_train[perms]

for client_id in range(num_clients):
    data_samples = int(x_train.shape[0] / num_clients)
    inds = (client_id * data_samples, (client_id + 1) * data_samples)
    x, y = x_train[inds[0]:inds[1]], y_train[inds[0]:inds[1]]
    np.save(f".././dataset_{client_id}", (x, y, x_test, y_test))

NameError: name 'x_test' is not defined